# **Detecção de Fraude em Cartão de Crédito**

- As empresas de cartão de crédito utilizam o aprendizado de máquina no momento da transação para determinar se devem aprovar ou recusar cobranças individuais.

## **1. Problema de Negócio**

- Neste projeto, nosso objetivo é criar um modelo  para detectar se um transação é um pagamento normal ou uma fraude, porém,  o foco principal é garantir que transações genuínas não sejam erroneamente identificadas como fraudulentas, evitando assim possíveis inconvenientes ou desconfortos para os clientes.

- O conjunto de dados utilizado é obtido do [Kaggle](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud). Este conjunto de dados contém transações reais feitas por Titulares de cartões de crédito europeus.

## **2. Preparando o ambiente para utilizar o PySpark**

### **Instalando o Java**

- O Apache Spark depende de outros sistemas, portanto, antes do Spark é preciso instalar as dependências. Primeiro, deve-se instalar o java

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

### **Instalando o Apache Spark**

- Em seguida, é preciso fazer o download do Spark, e como este roda sobre o Hadoop Distributed File System (HDFS), também é preciso fazer o download do Hadoop. Após os downloads, basta descompactar esses arquivos que o Spark já estará disponível no seu notebook Colab.

In [2]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

### **Configurando o ambiente**

- Precisamos dizer para o sistema onde encontrar o Java e o Spark, previamente instalados.

In [3]:
# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


- A seguir, vamos precisar da biblioteca findspark que vai nos permitir importar pacotes necessários para o funcionamento do pyspark.

In [4]:
# instalando a findspark
!pip install -q findspark

In [5]:
#importando a findspark
import findspark

# iniciando o findspark
findspark.init()

### **Iniciando o PySpark**


In [6]:
# importando o pacote necessário para iniciar uma seção Spark
from pyspark.sql import SparkSession

# iniciando o spark context
sc = SparkSession.builder.master('local[*]').getOrCreate()

# Verificando se a sessão foi criada
sc

## **3. Ingestão dos dados para o Apache Spark**

### **3.1 Habilitar o uso da API Kaggle no Google Colab**

In [7]:
# Instalando a biblioteca Kaggle
!pip install -q kaggle

- Faça o download do arquivo de configuração da API Kaggle chamado "kaggle.json" em sua conta Kaggle.

In [8]:
# Fazendo o upload do Token da API Kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"omarmarca","key":"9621522919e711e3ed6531006be9c535"}'}

In [9]:
# Movendo o arquivo kaggle.json para o diretório adequado
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
# Fazendo o download do  Conjunto de Dados
!kaggle datasets download -d mlg-ulb/creditcardfraud

 96% 63.0M/66.0M [00:02<00:00, 40.9MB/s]
100% 66.0M/66.0M [00:02<00:00, 30.3MB/s]


In [11]:
# Descompactando o Conjunto de Dados
!unzip creditcardfraud.zip

Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


## **3.2 Importar bibliotecas e conjunto de dados**

In [12]:
# importando as bibliotecas necessárias
from pyspark import *
from pyspark.sql import *

# carregando um conjunto de dados
credit = (sc.read
      .format('csv')
      .option('inferSchema',True)
      .option('delimiter',',')
      .option('header',True)
      .load('/content/creditcard.csv')
)

# Espiando o dataset
credit.show(10)

+----+------------------+-------------------+------------------+------------------+-------------------+-------------------+--------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|                V3|                V4|                 V5|                 V6|                  V7|                V8|                V9|                V10|               V11|               V12|               V13|               V14|               V15|                V16|                V17|                V18|                V19|                V20|                 V21|          

In [13]:
# Contando o número de linhas do dataset
credit.count()

284807

### Metadados:

>  - V1 até V28: as colunas possuem  informações não informativas  e contêm valores igualmente opacos.
>  - Time:contém os segundos decorridos entre cada transação e a primeira transação.
>  - Amount: é o valor da transação.
>  - Class: é a variável resposta e assume valor 1 em caso de fraude e 0
em caso contrário.

## **4. Análises preliminares**

In [14]:
# Verificando o Schema
credit.printSchema()

root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

In [15]:
# Quantas transações são fraudulentas e quantas não são?
credit.groupBy("Class").count().show()

+-----+------+
|Class| count|
+-----+------+
|    1|   492|
|    0|284315|
+-----+------+



- Das 284807 transações registradas no conjunto de dados, apenas 492 são identificadas como fraudulentas. Este desequilíbrio é significativo.

- O conjunto de dados possui 30 variáveis  features, e a variável Class é a variável target.

## **5. Pré-processamento**

### **Identificação de Dados Duplicados**

In [16]:
# Aplicando distinct() para remover linhas duplicadas
card=credit.distinct()
card.show(10)

+------+------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-----------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+------+-----+
|  Time|                V1|                 V2|                V3|                 V4|                V5|                V6|                 V7|                V8|                 V9|               V10|               V11|                V12|                V13|              V14|               V15|                V16|               V17|                V18|               V19|               V20|               V21|                V22

In [17]:
# Contando o número de linhas do dataset
card.count()

283726

In [18]:
# Quantas transações são fraudulentas e quantas não são?
card.groupBy("Class").count().show()

+-----+------+
|Class| count|
+-----+------+
|    1|   473|
|    0|283253|
+-----+------+



- Foram removidos 1081 registros duplicados.

### **Criando uma coluna com todas as preditoras no dataset**

In [19]:
# criando uma coluna com todas as preditoras
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Definindo as preditoras
def_preditoras = VectorAssembler(
    inputCols=[
    'V1',
    'V2',
    'V3',
    'V4',
    'V5',
    'V6',
    'V7',
    'V8',
    'V9',
    'V10',
    'V11',
    'V12',
    'V13',
    'V14',
    'V15',
    'V16',
    'V17',
    'V18',
    'V19',
    'V20',
    'V21',
    'V22',
    'V23',
    'V24',
    'V25',
    'V26',
    'V27',
    'V28',
    'Amount'
    ],
    outputCol= 'preditoras'
)

# criando uma coluna com todas as preditoras no dataset
card = def_preditoras.transform(card)

# Pegando somente os atributos que serao utilizados na modelagem
card_ml= card.select('Class', 'preditoras')

# Espiando dataset final
card_ml.show()

+-----+--------------------+
|Class|          preditoras|
+-----+--------------------+
|    0|[-0.5299122841865...|
|    0|[-0.6008163881153...|
|    0|[-1.5057791635308...|
|    0|[-0.4910030173022...|
|    0|[-0.5282175047788...|
|    0|[-3.9008102986784...|
|    0|[-1.5369580132194...|
|    0|[-1.2125020398473...|
|    0|[-0.8835285843058...|
|    0|[-1.4343476082208...|
|    0|[-0.7989760084090...|
|    0|[-0.6512154009528...|
|    0|[1.11534818942834...|
|    0|[1.00957555058854...|
|    0|[-2.2517410228050...|
|    0|[-0.4444871910125...|
|    0|[-1.3191686973385...|
|    0|[1.04406759129688...|
|    0|[0.89659392125538...|
|    0|[1.40127624693718...|
+-----+--------------------+
only showing top 20 rows



## **6. Treinamento dos modelos**

### **6.1 Dividindo o dataset entre treino e teste**

- O conjunto de dados é altamente desequilibrado e, para garantir que o conjunto de treino e o conjunto de teste preservem as porcentagens de classes, realizamos uma divisão estratificada personalizada e reservamos 20% dos dados para teste.

In [20]:
SPLIT_VALS = [0.8, 0.2]

# Split card_ml entre 0s e 1s
zeros = card_ml.filter(card_ml["Class"]==0)
ones = card_ml.filter(card_ml["Class"]==1)

  # Split em treinamento e teste
train0, test0 = zeros.randomSplit(SPLIT_VALS, seed=7)
train1, test1 = ones.randomSplit(SPLIT_VALS, seed=7)

  # Reunindo os conjuntos de dados novamente.
card_treino = train0.union(train1)
card_teste = test0.union(test1)

In [21]:
# Quantas transações são fraudulentas e quantas não são no conjunto de treinamento?
card_treino.groupBy("Class").count().show()

+-----+------+
|Class| count|
+-----+------+
|    1|   381|
|    0|226680|
+-----+------+



In [22]:
# Quantas transações são fraudulentas e quantas não são no conjunto de teste?
card_teste.groupBy("Class").count().show()

+-----+-----+
|Class|count|
+-----+-----+
|    1|   92|
|    0|56573|
+-----+-----+



### **6.2 Regressão Logística**

In [23]:
from pyspark.ml.classification import LogisticRegression

In [24]:
# definindo logist regression
rl= LogisticRegression(
    featuresCol='preditoras',
    maxIter = 5000,
    labelCol='Class',
    predictionCol='class_pred')

In [25]:
# Treinando o modelo
modelo_rl = rl.fit(card_treino)

In [26]:
# Ajustando o modelo aos dados de teste
modelo_rl_pred = modelo_rl.transform(card_teste)

In [27]:
# Selecionando aa colunas class_pred e Class
predita_observada = modelo_rl_pred.select('class_pred','Class')
predita_observada.show(10)

+----------+-----+
|class_pred|Class|
+----------+-----+
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
+----------+-----+
only showing top 10 rows



In [28]:
# Matriz de confusão modelo Regressão Logística
predita_observada\
  .crosstab('Class', 'class_pred')\
  .orderBy('Class_class_pred')\
  .show()

+----------------+-----+---+
|Class_class_pred|  0.0|1.0|
+----------------+-----+---+
|               0|56561| 12|
|               1|   37| 55|
+----------------+-----+---+



- O modelo de regressão logística identificou corretamente 56561 transações como legítimas ao mesmo tempo em que classificou erroneamente 12 transações legítimas como fraudulentas. Nos queremos
minimizar o último número porque não queremos incomodar os clientes recusando transações legítimas.

####  **Verificando as métricas da qualidade do ajuste do modelo**

In [29]:
# Calcular true positives, true negatives, false positives, false negatives

from pyspark.sql.functions import col

tp = modelo_rl_pred.filter((col("Class") == 1) & (col('class_pred') == 1)).count()
tn = modelo_rl_pred.filter((col("Class") == 0) & (col('class_pred') == 0)).count()
fp = modelo_rl_pred.filter((col("Class") == 0) & (col('class_pred') == 1)).count()
fn = modelo_rl_pred.filter((col("Class") == 1) & (col('class_pred') == 0)).count()

# Precisão
precision = tp / (tp + fp) if (tp + fp) != 0 else 0
print(f"Precision: {precision}",'\n')

# Recall= Sensitivity
recall = tp / (tp + fn) if (tp + fn) != 0 else 0.0
print(f"Recall: {recall}",'\n')

# Specificity
Specificity = tn / (tn + fp) if (tn + fp) != 0 else 0.0
print(f"Specificity: {Specificity}")

Precision: 0.8208955223880597 

Recall: 0.5978260869565217 

Specificity: 0.999787884679971


- Dado o interesse da empresa de cartão de crédito em garantir a satisfação contínua dos clientes e promover gastos, a especificidade é uma métrica crucial que avalia a capacidade de um teste em identificar corretamente amostras negativas, ou seja, em não rotular erroneamente transações legítimas como fraudulentas.

- Vamos ver se o  algoritmo Random forest pode aprimorar  a  especificidade.

### **6.3 Random Forest**

In [30]:
from pyspark.ml.classification import RandomForestClassifier

In [31]:
# definindo a floresta
rf= RandomForestClassifier(
    featuresCol='preditoras',
    labelCol='Class',
    predictionCol='class_pred',
    numTrees=50,
    maxDepth=30
    )

In [32]:
# Treinando a floresta
modelo_rf = rf.fit(card_treino)

In [33]:
# Ajustando a floresta aos dados de teste
modelo_rf_pred = modelo_rf.transform(card_teste)

In [34]:
# Selecionando as colunas class_pred e Class
predita_observada1 = modelo_rf_pred.select('class_pred','Class')
predita_observada1.show(10)

+----------+-----+
|class_pred|Class|
+----------+-----+
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
+----------+-----+
only showing top 10 rows



In [35]:
# Matriz de confusão
predita_observada1\
  .crosstab('Class', 'class_pred')\
  .orderBy('Class_class_pred')\
  .show()

+----------------+-----+---+
|Class_class_pred|  0.0|1.0|
+----------------+-----+---+
|               0|56564|  9|
|               1|   24| 68|
+----------------+-----+---+



- O modelo random forest confundiu apenas nove transações legítimas como fraudulentas. Isso é um melhoria em relação à regressão logística.
- O modelo capturou cerca de 74% das transações fraudulentas.

#### **Verificando as métricas da qualidade do ajuste do modelo Random Forest**

In [36]:
# Calcular true positives, true negatives, false positives, false negatives

tp1 = modelo_rf_pred.filter((col("Class") == 1) & (col('class_pred') == 1)).count()
tn1 = modelo_rf_pred.filter((col("Class") == 0) & (col('class_pred') == 0)).count()
fp1 = modelo_rf_pred.filter((col("Class") == 0) & (col('class_pred') == 1)).count()
fn1 = modelo_rf_pred.filter((col("Class") == 1) & (col('class_pred') == 0)).count()

# Precisão
precision1 = tp1 / (tp1 + fp1) if (tp1 + fp1) != 0 else 0
print(f"Precision: {precision1}",'\n')

# Recall= Sensitivity
recall1 = tp1 / (tp1 + fn1) if (tp1 + fn1) != 0 else 0.0
print(f"Recall: {recall1}", '\n')

# Specificity
Specificity1 = tn1 / (tn1 + fp1) if (tn1 + fp1) != 0 else 0.0
print(f"Specificity: {Specificity1}")

Precision: 0.8831168831168831 

Recall: 0.7391304347826086 

Specificity: 0.9998409135099783


- Em comparação com o modelo de regressão logística, este modelo apresenta um desempenho ligeiramente superior.

## **7. Conclusão**

- O modelo Random Forest demonstra uma precisão satisfatória na detecção de fraudes de cartão de crédito, utilizando a métrica de avaliação de Especificidade.

- Podemos aprimorar a previsão do modelo através da utilização da técnica de combinação de modelos conhecida como "Stacking ensemble".





